In [1]:
def deep_dive(manufact,target,reference):
    
    # Importing required library
    import pandas as pd
    
    # Reading the excel file
    df = pd.read_excel('Case Study - Deep Dive Analysis.xlsx')
    
    # taking out month and year from the date 
    df['month_num'] = pd.DatetimeIndex(df['month']).month
    df['year']  = pd.DatetimeIndex(df['month']).year
    
    # extracting month and year from user input
    target_month = target[:3].title()
    reference_month = reference[:3].title()
    target_year = pd.to_numeric(target[3:])
    reference_year = pd.to_numeric(reference[3:])
    
    # dictionary to map month
    mapped_months = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}

    # mapping month to it's numeral
    month_target_period = [value for key, value in mapped_months.items() if target_month == key][0]
    month_reference_period = [value for key, value in mapped_months.items() if reference_month == key][0]
    
    # selecting data as per target and reference window
    df1 = df.query('Manufacturer == @manufact.upper() & month_num <= @month_target_period & year <= @target_year & month_num >= @month_reference_period & year >= @reference_year')
    
    # creating a grouped table
    temp = df1.groupby('month').sum()
    
    # grouped on geo and product level
    temp1  = df1.groupby(['month','Zone','Region','Brand','Subbrand'], as_index=False).sum()
    
    # creating grouped tables for all the levels
    zone_group = temp1.groupby(['month', 'Zone'],as_index=False).sum()
    region_group = temp1.groupby(['month','Region'],as_index=False).sum()
    brand_group = temp1.groupby(['month', 'Brand'],as_index=False).sum()
    subbrand_group = temp1.groupby(['month', 'Subbrand'],as_index=False).sum()
    
    # list to store the results
    lev = []
    
    # calculating Growth_Rate, Contribution for Zone, Region, Brand and Subbrand levels respectively
    for i in range(0, len(zone_group.index)//2):
        for j in range(0, len(zone_group.index)):
            if (zone_group['month'][i] != zone_group['month'][j] and zone_group['Zone'][i] == zone_group['Zone'][j]):
                lev.append(['Zone',zone_group['Zone'][i],(zone_group['Value Offtake(000 Rs)'][j] - zone_group['Value Offtake(000 Rs)'][i]),(zone_group['Value Offtake(000 Rs)'][j]*100)/zone_group['Value Offtake(000 Rs)'][(len(zone_group.index)//2):].sum()])
                
    for i in range(0, len(region_group.index)//2):
        for j in range(0, len(region_group.index)):
            if (region_group['month'][i] != region_group['month'][j] and region_group['Region'][i] == region_group['Region'][j]):
                lev.append(['Region',region_group['Region'][i],(region_group['Value Offtake(000 Rs)'][j] - region_group['Value Offtake(000 Rs)'][i]),(region_group['Value Offtake(000 Rs)'][j]*100)/region_group['Value Offtake(000 Rs)'][(len(region_group.index)//2):].sum()])
                
    for i in range(0, len(brand_group.index)//2):
        for j in range(0, len(brand_group.index)):
            if (brand_group['month'][i] != brand_group['month'][j] and brand_group['Brand'][i] == brand_group['Brand'][j]):
                lev.append(['Brand',brand_group['Brand'][i],(brand_group['Value Offtake(000 Rs)'][j] - brand_group['Value Offtake(000 Rs)'][i]),(brand_group['Value Offtake(000 Rs)'][j]*100)/brand_group['Value Offtake(000 Rs)'][(len(brand_group.index)//2):].sum()])
                
    for i in range(0, len(subbrand_group.index)//2):
        for j in range(0, len(subbrand_group.index)):
            if (subbrand_group['month'][i] != subbrand_group['month'][j] and subbrand_group['Subbrand'][i] == subbrand_group['Subbrand'][j]):
                lev.append(['Subbrand',subbrand_group['Subbrand'][i],(subbrand_group['Value Offtake(000 Rs)'][j] - subbrand_group['Value Offtake(000 Rs)'][i]),(subbrand_group['Value Offtake(000 Rs)'][j]*100)/subbrand_group['Value Offtake(000 Rs)'][(len(subbrand_group.index)//2):].sum()])
        
    # creating a dataframe to list Manufacturer, Level, Focus_Area, Growth_Area, Contribution and Product
    output = pd.DataFrame(lev, columns=['Level','Focus_Area','Growth_Rate','Contribution'])
    output.insert(0,'Manufacturer',manufact.upper())
    output['Product'] = (output['Growth_Rate'] * output['Contribution'])/10000
    output = output.round(2)
    
    # Final call to return the appropriate results
    if (temp.iloc[0][0] - temp.iloc[1][0]) == 0:
        return(f'There is no drop in the sales for {a} in the {c} - {b}')
    else:
        return(output)

In [2]:
print(deep_dive('nestle','may2019','apr2019'))

  Manufacturer     Level      Focus_Area  Growth_Rate  Contribution  Product
0       NESTLE      Zone            East       656.63         29.79     1.96
1       NESTLE      Zone           North       -59.31         30.87    -0.18
2       NESTLE      Zone           South        65.02         13.84     0.09
3       NESTLE      Zone            West        79.55         25.50     0.20
4       NESTLE    Region           Rural        43.66         25.21     0.11
5       NESTLE    Region           Urban       698.24         74.79     5.22
6       NESTLE     Brand            MILO       741.89        100.00     7.42
7       NESTLE  Subbrand  MILO ACTIVE GO       741.89        100.00     7.42
